<a href="https://colab.research.google.com/github/scottthomaswx/model-analysis/blob/main/model_events.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install openpyxl
!pip install pandas
!pip install numpy
!pip install dateutil

from openpyxl import load_workbook
from openpyxl import Workbook
from openpyxl import utils as utils
import pandas as pd
from dateutil.parser import *
from datetime import datetime, timedelta
import numpy

In [ ]:
rawstart = '12/28/2025 1200 UTC'
rawend = '12/29/2025 1200 UTC'

In [ ]:
startdates = []
enddates = []

In [ ]:
startdates=[]
startdates.append(parse(rawstart))
enddates.append(parse(rawend))

In [ ]:
tdelta = [None]*len(startdates)
x = 0
while x < len(startdates):
  tdelta[x] = int((enddates[x].replace(tzinfo=None)-startdates[x].replace(tzinfo=None)).total_seconds()/10800)
  x += 1

In [ ]:
alldates = numpy.empty((max(tdelta[:])+1), dtype=object)

In [ ]:
y = 0
i = 0
j = 0

while j <= 8:
    alldates[j] = numpy.datetime64(startdates[y]) + numpy.timedelta64(i,'h')
    i += 3
    j += 1


/tmp/ipython-input-2220523698.py:6: UserWarning: no explicit representation of timezones available for np.datetime64
  alldates[j] = numpy.datetime64(startdates[y]) + numpy.timedelta64(i,'h')


In [ ]:
finaldates = numpy.empty((max(tdelta[:])+1, 81), dtype=object)

In [ ]:
i=0
z=0

while i <= 8:
    z = 0
    while z < 81:
      finaldates[i,z] = numpy.datetime64(alldates[i]) - 3*(numpy.timedelta64(z,'h'))
      z += 1
    i += 1

In [ ]:
wb = Workbook()

In [ ]:
date=[None]*len(startdates)

In [ ]:
for i in range(0,len(startdates)):
  if i == 0:
    ws=wb.active
    date[i]=startdates[i].strftime("%Y-%m-%d-%H")
    ws.title=str(date[i])
  else:
    date[i]=startdates[i].strftime("%Y-%m-%d-%H")
    wb.create_sheet(title=str(date[i]),index=i)

In [ ]:
s = 0

In [ ]:
def as_text(value):
    if value is None:
        return ""
    return str(value)

In [ ]:
for sheet in wb:
    colheader = [u' ', u'F00', u'F03', u'F06', u'F09', u'F12', u'F15', u'F18', u'F21', u'F24', u'F27', u'F30',\
                 u'F33', u'F36', u'F39', u'F42', u'F45', u'F48', u'F51', u'F54', u'F57', u'F60', u'F63', u'F66',\
                u'F69', u'F72', u'F75', u'F78', u'F81', u'F84', u'F87', u'F90', u'F93', u'F96', u'F99', u'F102',\
                u'F105', u'F108', u'F111', u'F114', u'F117', u'F120', u'F123', u'F126', u'F129', u'F132', u'F135',\
                u'F138', u'F141', u'F144', u'F147', u'F150', u'F153', u'F156', u'F159', u'F162', u'F165', u'F168',\
                 u'F171', u'F174', u'F177', u'F180', u'F183', u'F186', u'F189', u'F192', u'F195', u'F198', u'F201',\
                 u'F204', u'F207', u'F210', u'F213', u'F216', u'F219', u'F222', u'F225', u'F228', u'F231', u'F234',\
                 u'F237', u'F240']
    for j in range(1,82):
        sheet.cell(row=1,column=j+1).value = colheader[j]
    for i in range(1,10):
        sheet.cell(row=i+1,column=1).value = i

    for i in range(1,10):
        for j in range(1,82):
            if str(finaldates[i-1,j-1]) != 'None':
                sheet.cell(row=i+1,column=j+1).value = pd.to_datetime(str(finaldates[i-1][j-1])).strftime('%y%m%d/%H%M') + 'F0' + str(format(3*(j-1), '02d'))
    s += 1

    dims = {}
    for row in sheet.rows:
        for cell in row:
            if cell.value:
                dims[cell.column] = max((dims.get(cell.column, 0), len(str(cell.value))))

    for column_cells in sheet.columns:
        length = max(len(as_text(cell.value)) for cell in column_cells)
        sheet.column_dimensions[utils.get_column_letter(column_cells[0].column)].width = length


In [ ]:
wb.save(filename='scott_Dec25.xlsx')